In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_chan, out_chan, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_chan, out_chan, kernel_size=3, padding=1, stride=stride)
        # conv2    input_shape = output_shape
        self.conv2 = nn.Conv2d(out_chan, out_chan, kernel_size=3, padding=1, stride=1)
        # res_conv (kernel=1, padding=0, stride=input_stride): x - 1 + 2*0 + stride = x - 1 + stride
        # conv1    (kernel=3, padding=1, stride=input_stride): x - 3 + 2*1 + stride = x - 1 + stride
        self.res_conv = nn.Conv2d(in_chan, out_chan, kernel_size=1, stride=stride)

        self.bn1 = nn.BatchNorm2d(out_chan)
        self.bn2 = nn.BatchNorm2d(out_chan)
        self.in_chan = in_chan
        self.out_chan = out_chan
        self.stride = stride

    def forward(self, x):
        # conv_bn_relu
        y = F.relu(self.bn1(self.conv1(x)))
        # conv_bn_res_relu
        # residual connect:
        # case 1: if in_chan != out_chan or stride != 1, use 1*1 conv
        # case 2: else, use x itself
        y = F.relu(self.bn2(self.conv2(y)) 
                   + self.res_conv(x) if self.in_chan != self.out_chan or self.stride != 1 else x)
        return y

def residual_module(in_chan, out_chan, num_layers, first_block=False):
    layers = []
    for i in range(num_layers):
        if i == 0 and not first_block:
            layers.append( ResidualBlock(in_chan, out_chan, 2) )
        else:
            layers.append( ResidualBlock(in_chan, out_chan) )
        in_chan = out_chan
    return layers


stage1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.BatchNorm2d(64), nn.ReLU(),
                       nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
stage2 = nn.Sequential(*residual_module(64, 64, 2, True))
stage3 = nn.Sequential(*residual_module(64, 128, 2))
stage4 = nn.Sequential(*residual_module(128, 256, 2))
stage5 = nn.Sequential(*residual_module(256, 512, 2))
stage6 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(512, 10))

resnet = nn.Sequential(stage1, stage2, stage3, stage4, stage5, stage6)

#print('resnet: ')
#x = torch.randn((3,1,224,224))
#for layer in resnet:
#    x = layer(x)
#    print('%-10s: %-30s'%(layer.__class__.__name__, x.shape))

In [12]:
import torchvision
import torch
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-3)

optimizer.state_dict()['param_groups'][0]['lr']




0.1

In [13]:
import pandas as pd
from tqdm.notebook import tqdm
import time

tqdm.pandas('test')
_ = df.progress_apply(lambda _: time.sleep(0.2), axis=2)


AttributeError: 'FastAPI' object has no attribute 'debug'